In [1]:
pip install google-cloud-storage

  Using cached google_cloud_storage-2.18.2-py2.py3-none-any.whl.metadata (9.1 kB)
  Using cached google_cloud_core-2.4.1-py2.py3-none-any.whl.metadata (2.7 kB)
  Using cached google_resumable_media-2.7.2-py2.py3-none-any.whl.metadata (2.2 kB)
  Using cached proto_plus-1.24.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached cachetools-5.5.0-py3-none-any.whl.metadata (5.3 kB)
  Using cached rsa-4.9-py3-none-any.whl.metadata (4.2 kB)
Using cached google_cloud_storage-2.18.2-py2.py3-none-any.whl (130 kB)
Using cached google_cloud_core-2.4.1-py2.py3-none-any.whl (29 kB)
Using cached google_resumable_media-2.7.2-py2.py3-none-any.whl (81 kB)
Using cached cachetools-5.5.0-py3-none-any.whl (9.5 kB)
Using cached proto_plus-1.24.0-py3-none-any.whl (50 kB)
Using cached rsa-4.9-py3-none-any.whl (34 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [13]:
from io import BytesIO
from google.cloud import storage

def list_buckets():
    storage_client = storage.Client()
    buckets = list(storage_client.list_buckets())
    print(buckets)
    
def upload_blob(bucket_name, source_file_name, destination_blob_name):
    storage_client = storage.Client()
    
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    # Optional: set a generation-match precondition to avoid potential race conditions
    # and data corruptions. The request to upload is aborted if the object's
    # generation number does not match your precondition. For a destination
    # object that does not yet exist, set the if_generation_match precondition to 0.
    # If the destination object already exists in your bucket, set instead a
    # generation-match precondition using its generation number.
    generation_match_precondition = 0

    blob.upload_from_filename(source_file_name, if_generation_match=generation_match_precondition)
    print(
        f"File {source_file_name} uploaded to {destination_blob_name}."
    )
    
def upload_image(bucket_name, content, destination_blob_name):
    storage_client = storage.Client()
    
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    
    res = blob.upload_from_file(BytesIO(content), content_type="image/jpeg")
    print(f"res: {res}")
    

In [6]:
import os
import requests

GOOGLE_MAPS_API_KEY = os.getenv("GOOGLE_MAPS_API_KEY")

def get_place_details(place_name, place_address):
    search_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"
    params = {
        'input': f"{place_name}, {place_address}",
        'inputtype': 'textquery',
        'fields': 'place_id',
        'key': GOOGLE_MAPS_API_KEY
    }

    response = requests.get(search_url, params=params)
    result = response.json()

    if response.status_code == 200:
        candidates = result.get('candidates', [])

        if candidates:
            place_id = candidates[0]['place_id']

            # 장소의 세부 정보를 가져오는 API 요청
            details_url = "https://maps.googleapis.com/maps/api/place/details/json"
            details_params = {
                'place_id': place_id,
                'fields': 'photos,name,rating,user_ratings_total',
                'key': GOOGLE_MAPS_API_KEY
            }

            details_response = requests.get(details_url, params=details_params)
            details_result = details_response.json()

            # details_result 반환 및 JSON 저장
            return details_result
    return None

In [7]:
import pandas as pd

csv_file_path = '../data/KC_MTPCLT_RSTRNT_DATA_2023.csv'  # 실제 CSV 파일 경로를 입력하세요
df = pd.read_csv(csv_file_path)

df = df[df["PET_POSBL_AT"] == "Y"]
df.head(5)

,FCLTY_NM,CTGRY_ONE_NM,CTGRY_TWO_NM,CTGRY_THREE_NM,CTPRVN_NM,SIGNGU_NM,LEGALDONG_NM,LI_NM,LNBR_NO,ROAD_NM,...,WKEND_OPER_TIME_DC,FRE_PARKNG_AT,VALET_PARKNG_POSBL_AT,INFN_CHAIR_LEND_POSBL_AT,WCHAIR_HOLD_AT,PET_POSBL_AT,VGTR_MENU_HOLD_AT,HALAL_FOOD_HOLD_AT,GFRE_FOOD_HOLD_AT,LAST_UPDT_DE
129,거루캥테이블,음식점/유흥시설,오세아니아음식,호주 아시아,부산광역시,해운대구,송정동,NaN,127-6 번지,송정중앙로5번길,...,토-일 09시30분-21시,Y,N,N,N,Y,N,N,N,20221130
133,게리가,음식점/유흥시설,동남아시아음식,기타,제주특별자치도,서귀포시,남원읍,신례리,71-3 번지,공천포로,...,토-일 10시-18시,Y,N,N,N,Y,N,N,N,20221130
163,고메303,음식점/유흥시설,유럽음식,이탈리아,경상남도,양산시,물금읍,가촌리,705-3 번지,가촌로,...,토-일 11시30분-21시30분,Y,N,N,N,Y,N,N,N,20221130
199,골드홀릭,음식점/유흥시설,동아시아음식,일본,인천광역시,중구,북성동1가,NaN,98-12 번지,월미문화로,...,토-일 11시-24시,Y,N,N,N,Y,N,N,N,20221130
201,골든타이,음식점/유흥시설,동남아시아음식,태국,서울특별시,송파구,방이동,NaN,171-4 번지,위례성대로,...,토-일 11시-22시,Y,N,N,N,Y,N,N,N,20221130


In [14]:
for i in range(5):
    place_name = df.iloc[i]["FCLTY_NM"]
    place_address = df.iloc[i]["RDNMADR_NM"]
    details = get_place_details(place_name, place_address)
    if details['result']['photos']:
        photo_reference = details['result']['photos'][0]['photo_reference']
        photo_url = f"https://maps.googleapis.com/maps/api/place/photo?maxwidth=400&photoreference={photo_reference}&key={GOOGLE_MAPS_API_KEY}"
        response = requests.get(photo_url)
        if response.status_code == 200:
            upload_image("pcp-places", response.content, f"{place_name}.jpg")
    

res: None
res: None
res: None
res: None
res: None
